In [ ]:
class Agent:
    def __init__(self, name, current_position, target_position, next_position):
        self.name = name
        self.current_position = current_position
        self.target_position = target_position
        self.next_position = next_position
        
    def describe(self):
        print(f"Agent {self.name} is at {self.current_position} and wants to go to {self.target_position}. The next position is {self.next_position}.")